## Student Learning

This notebook will explore data on student learning. 
More specifically, it will explore data regarding:

- which course was seen and for how long
- which student saw which course
- Date on which the courses were watched

In [7]:

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [8]:
student_learning = pd.read_csv('dataSets/365_database/365_student_learning.csv')
student_learning.head()

,student_id,course_id,minutes_watched,date_watched
0,258798,23,0.3,2022-01-01
1,258800,2,12.9,2022-04-01
2,258800,7,46.8,2022-01-03
3,258800,7,4.3,2022-01-04
4,258800,7,31.7,2022-01-07
